In [1]:
from src import pdf_to_markdown_with_ocr
from src import read_txt_file
from src import split_text_into_chunks
from src import FORM_GPT, AMA_GPT, PRO_GPT, HAM_GPT, SOL_GPT, VAL_GPT

In [2]:
def GPT_main(target_element):
    try:
        AMA_response = AMA_GPT(target_element)
        PRO_response = PRO_GPT(AMA_response, target_element)
        HAM_response = HAM_GPT(target_element, AMA_response, PRO_response)

        SOL_response = SOL_GPT(HAM_response[0]['Question'])
        VAL_response = VAL_GPT(f"{SOL_response}")

        valid_list = [True, 'True', False, 'False']

        if VAL_response in valid_list:
            return HAM_response
        else:
            return None

    except Exception as e:
        print('='*50)
        print(f"[에러 발생] | {e} | {target_element}")
        print('='*50)

##### ./source_dataset 폴더 내에 있는 pdf 기반 데이터셋 생성

In [6]:
import os

file_list = [f"./source_dataset/crawled/{pdf}" for pdf in os.listdir('./source_dataset/crawled/')]

In [ ]:
final_list = []

# file 선택
for i in range(len(file_list)):
    target_file = file_list[i]
    # txt 파일 읽어오기
    text = read_txt_file(target_file)
    chunk_list = split_text_into_chunks(text)

    current_file_QA = []

    # file 내 청크 선택
    for j in range(len(chunk_list)):
        target_chunk = chunk_list[j]
        formed_dict = FORM_GPT(target_chunk)

        # 청크 기반으로 생성된 마크다운 형식의 문장 선택
        idx = 0
        for item in formed_dict:
            print('='*50)
            try:
                idx += 1
                target_element = formed_dict[item]

                # GPT 기반으로 QA쌍 생성
                QA_pairs = GPT_main(target_element)

                if QA_pairs:
                    for QA_pair in QA_pairs:
                        current_file_QA.append(QA_pair)
                        print(QA_pair)
                    print(f"[성공] ({i+1}/{len(file_list)}번 파일) | ({j+1}/{len(chunk_list)}번 chunk) | ({idx}/{len(formed_dict)}번 element)")
                else:
                    print(f"[실패] ({i+1}/{len(file_list)}번 파일) | ({j+1}/{len(chunk_list)}번 chunk) | ({idx}/{len(formed_dict)}번 element)")

            except Exception as e:
                print(f"[에러] ({i+1}/{len(file_list)}번 파일) | ({j+1}/{len(chunk_list)}번 chunk) | ({idx}/{len(formed_dict)}번 element)")
                print(e)

    for QA in current_file_QA:
        print('-'*50)
        print(QA)
        print('-'*50)
        final_list.append(QA)